# The Questions

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


columns = pd.read_csv("recipeBool.csv").columns

## Bei Einrichtung
1. Frage: Willst du einen Thermomix verwenden?
2. Frage: Nenne Zutaten, welche du nicht verwenden möchtest.
3. Frage: Hast du irgendwelche Alergien?
4. Frage: Besondere Ernährungsweise?
5. Frage: Wie bewertest du folgende 5 Gerichte? (0-5)


Funktion zu Erstellung eines Userprofiles:

In [15]:
#creating helping df´s for finden id´s

con = sqlite3.connect("RecipeDB.db")
ing_df = pd.read_sql("Select * FROM IngFamily", con)
temp = pd.read_sql('SELECT * FROM Ingredients',con)
ing_df.index = ing_df["familyID"]
for x in temp.itertuples():
    ing_df.loc[x.familyID]['familyName']+= ' '+x.ingName
ing_df = ing_df.drop(["familyID"], axis = 1)

ing_df.rename(columns = {'familyName':'info'}, inplace = True)

tags_df = pd.read_sql("Select * from Tags", con)
tags_df.index = tags_df["tagID"]
tags_df['info'] = tags_df['tagName']+' '+tags_df['tagType']
tags_df = tags_df.drop(["tagID"], axis = 1)
tags_df = tags_df.drop(["tagName"], axis = 1)
tags_df = tags_df.drop(["tagType"], axis = 1)

alg_df = pd.read_sql("Select * from Allergens", con)
alg_df.index = alg_df["alleID"]
alg_df['info']= alg_df['alleType']+' '+alg_df['alleName']+' '+alg_df['alleSlug']
alg_df = alg_df.drop(["alleID"], axis = 1)
alg_df = alg_df.drop(["alleType"], axis = 1)
alg_df = alg_df.drop(["alleName"], axis = 1)
alg_df = alg_df.drop(["alleSlug"], axis = 1)

thermo = tags_df[tags_df["info"].str.contains("thermomix")]




In [12]:
alg_df.head()

,info
alleID,
57962a07b7e8697d4b3052f5,gluten Weizen weizen
57962a07b7e8697d4b3052f3,maycontaintracesofallergens kann Spuren von Al...
57962a07b7e8697d4b3052fa,milk Milch laktose
57962a07b7e8697d4b3052fd,mustard Senf senf
57962a07b7e8697d4b3052f6,egg Ei ei


In [16]:
ing_df

,info
familyID,
58877cf3ed71a76e9c03fc1c,Reis Arborio-Reis Basmatireis Jasminreis Gerst...
58877cf3ed71a76e9c03fb54,Brot Arabisches Fladenbrot Naan-Brot Wraps Bag...
58877cf3ed71a76e9c03fc4e,Essig Apfelessig Essig* Condimento-Bianco-Essi...
58877cf3ed71a76e9c03fb3b,Aubergine Aubergine
58877cf3ed71a76e9c03fc47,Truthahn Putenbrust Putenbrust-Aufschnitt Pute...
...,...
58877cf3ed71a76e9c03fc3d,Karamellbonbons Karamellbonbons
58877cf3ed71a76e9c03fc59,Schinken Schwarzwälder Schinken
58877cf3ed71a76e9c03fc5a,Gebratene Zwiebel Röstzwiebeln Röstzwiebeln


In [17]:
tags_df.head()

,info
tagID,
57d13370b7e8697d4b30531a,leichter Genuss low-calorie
57d13370b7e8697d4b305319,scharf spicy
58aeb96c99052d5fee3dd712,Chef's Choice chefs-choice
57d13370b7e8697d4b305317,Vegetarisch vegetarian
57d13370b7e8697d4b30530e,Zeit sparen under-30-minutes


In [19]:
test = (False, ["Schwein", "Tomaten", "Schimmelkäse"], ["Laktose", "Schwefel"], ["Proteine", "fettarm"])
def create_Userprofile(tmix, ing, alg, tags):
    data = np.full([1, len(columns)], 0.5, dtype=np.float64)
    user_Profile = pd.DataFrame(data=data, columns = columns)
    user_Profile = user_Profile.drop(["Unnamed: 0"], axis = 1)
    for x in thermo.itertuples():
        user_Profile[x.Index] = 1.0 if tmix else 0.0
    for x in ing:
        for i in ing_df[ing_df['info'].str.contains(x)].itertuples():
            #TODO sachen wie gewürzmischungen oder komplimentär zutaten werden auch gefunden 
            print('ing :',x,' id+ info', i.Index, ' ', i.info)
    return user_Profile

create_Userprofile(test[0], test[1], test[2], test[3])
    
    
    
    
    

ing : Schwein  id+ info 58877cf3ed71a76e9c03fc12   Schweinefleisch Schweinefilet Speckwürfel Schinkenspeck Bacon Schweineschnitzel Chorizo Schweinenacken Schweinefleisch Schweinerückensteak Schweinelachssteaks Pancetta Pulled Pork Schweinbauch Vegankebab-Döner Prosciutto Schweinelachs gewürfelter Bacon Truthahn-Bacon Rohschinken Serranoschinken Schweinebraten Krustenbraten Schinkenbraten
ing : Schwein  id+ info 58877cf3ed71a76e9c03fc32   Gewürzmischung Gewürzmischung "Linsen-Suppe" kleine Gewürzgurke Cujan-Gewürzmischung Gewürzmischung "Cafè de Paris" Bratkartoffel-Gewürz Getrocknetes Bohnenkraut Gyros-Gewürzmischung Gewürzmischung "Paprikapulver/Kurkuma" Gewürzmischung "Rauchige Gemüsepfanne" Gewürzmischung "Pizzaschnecken" Gewürzmischung "Rauchiger Paneer" Gewürzmischung "Putengeschnetzeltes" Gewürzmischung "Pommes frites" Gewürzmischung "Gemüse-Couscous" Gewürzmischung "Bratkartoffel" Gewürzmischung "Tofu-Wrap" Gewürzmischung "Linguine-Soße" Gewürzmischung "Hähnchen" Gewürzmischung 

,57d13370b7e8697d4b30530e,57d13370b7e8697d4b30530f,57d13370b7e8697d4b305311,57d13370b7e8697d4b305313,57d13370b7e8697d4b305314,57d13370b7e8697d4b305315,57d13370b7e8697d4b305316,57d13370b7e8697d4b305317,57d13370b7e8697d4b305318,57d13370b7e8697d4b305319,...,57962a07b7e8697d4b3052f7,57962a07b7e8697d4b3052f8,57962a07b7e8697d4b3052f9,57962a07b7e8697d4b3052fa,57962a07b7e8697d4b3052fb,57962a07b7e8697d4b3052fc,57962a07b7e8697d4b3052fd,57962a07b7e8697d4b3052fe,57962a07b7e8697d4b3052ff,5a95b0bbae08b555554f4b52
0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


## Bei Normaler Verwendung
### 1. Option:
Empfehlung eines Rezeptes

### 2. Option:
Spezifischere Empfehlung durch folgende Frage:
1. Nenne Zutaten, welche du gerne verwenden möchtest.